## Developing Spark Applications: PairRDDFunctions

In this lab, you will take your first steps using PairRDDFunctions using Jupyter Notebook.

### Objectives

Use the SparkContext to bootstrap RDDs of Tuples in order to use PairRDDFunctions. Use the PairRDDFunctions to more easily calculate values in the RDD.

### Consider Tuples

Scala's support for arbitrary pairs (and triplets, quadruplets, and so on) of data is reflected in Spark. If an RDD's element is of a Tuple type, then, through Spark implicits, that RDD gets enriched with PairRDDFunctions, which provide convenient means of performing calculations on the Tuples.


## Instructions

In order to work with data using Spark, we need to get our hands on a dataset.  Spark calls such a dataset a "resilient distributed dataset", or `RDD`.  There are several ways to get your hands on an `RDD`.  In most cases, however, you're going to use a `SparkContext` to do that.

Let's start with something simple:  a plain, ol' Scala collection of the integers from 1 to 10,000.


### Get a SparkSession

In order to work with Spark's SQL support, we need to first get our hands on a special context called `SparkSession`.  
The SparkSession class is the entry point into all functionality in Spark. 

> Note: as of Spark 2.0, SparkSession replaced SqlContext. However, we could still use SqlContext as it's being kept for backward compatibility.

We'll use SparkSession.builder to create a SparkSession. SparkSession.builder lets you define you application name and it also lets you set various parameters in the Spark config, although there is no need to do so for our simple example.

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Python Pair RDD Functions").getOrCreate()

Check if Spark Context works

Create an RDD consisting of numbers from 0 to 999. Take a random sample of 5 numbers (without replacement - first parameter to takeSample function).

You should see the output consisting of 5 random numbers from the initial range.

In [ ]:
rdd = spark.sparkContext.parallelize(range(1000))

rdd.takeSample(False, 5)

### Read external file into RDD

You'll be using a file sherlock-holmes.txt. You need to retrieve it from the container folder (/home/jovyan/Resources) mounted to host machine directory - see instructions about setting up the Docker container to run Jupyter Notebook.

Read text into RDD. 

Make sure that the file was read correctly by printing the RDD element count (here the number of lines in the file).

In [ ]:
file = "/home/jovyan/Resources/sherlock-holmes.txt"

lines = spark.sparkContext.textFile(file)

print(lines.count())


### Count the words

Before we get to controlling the degree of parallelism let's make sure we're able to count the words and find the one with the maximum count.

Split the lines into individual words, for each word make a tuple, where the first element is the word itself and the second element is the initial count 1. Finally reduce by key obtaining the RDD with unique words and the total counts.


In [ ]:
import re

words = lines.flatMap(lambda w : re.split('[ \],.:;?!\-@#\(\)\\\*\"\/]*', w)).map(lambda w : w.lower())

words.take(10)


We'll consider two equivalent ways to accomplish our goal
They will be using PairRDDFunctions, which operate on data organized as key-value pairs. The functions typically agregate values for the keys.

### <font color=blue>Use reduceByKey</font>

One of the most useful PairRDDFunctions is reduceByKey, which transforms a pair RDD anto another pair RDD, where the values for each key are aggregated using the supplied reduce function.

In our case, the pair's key is the word, and the value is the count. The initial value for each word is 1.

The reduce function is a simple addition as we'll be adding the counts of the same words. The values will be aggregated, finally resulting in the total count for each distinct word.

Let's transform our words RDD into initial pairs RDD and while we're at it convert words into lower case for better accuracy of the counts.

Next we'll be applying reduceByKey function yielding the total word counts.



In [ ]:
from operator import add

pairs = words.map(lambda w : (w, 1))

counts = pairs.reduceByKey(add)

counts.take(10)

#### Convert RDD into Map

In [ ]:
map = counts.collectAsMap()

print("if  count " + str(map["if"]))
print("and count " + str(map["and"]))
print("but count " + str(map["but"]))


### <font color=blue>Use countByKey</font>

It just so happens that PairRDDFunctions has a method called `countByKey`, which returns a Map of counts of the occurrence of each key among the pairs. 

This function also requires a pair RDD. However, since it counts the occurrences of the keys it doesn't really matter what the values are (even the value type is irrelevant). Therefore we'll be using None as opposed to 1 as the initial value of the pairs.

Note that unlike `reduceByKey` there is no need to collect RDD values into a Map as `countByKey` does it automatically.

In [ ]:
pairs = words.map(lambda w : (w, None))

counts = pairs.countByKey()

print("if  count " + str(counts["if"]))
print("and count " + str(counts["and"]))
print("but count " + str(counts["but"]))


### Other interesting things about Sherlock Holmes

#### Find the word with the highest count

We'll use `reduce` function, which out of two pair elements returns the one with the higher count (note the way we access the individual elements of pairs).



In [ ]:
def getMaxCount(r, c):
  if (r[1] > c[1]):
    return r
  else:
    return c

pairs = words.map(lambda w : (w, 1))
counts = pairs.reduceByKey(add)

max = counts.reduce(lambda r, c: getMaxCount(r, c))

print("word with max count " + str(max))


#### Find the longest word

We'll use `reduce` function, which out of two pair elements returns the one with the longer key.


In [ ]:
def getMaxLength(r, c):
  if (len(r[0]) > len(c[0])):
    return r
  else:
    return c

pairs = words.map(lambda w : (w, 1))
counts = pairs.reduceByKey(add)

max = counts.reduce(lambda r, c: getMaxLength(r, c))

print("word with max count " + str(max))


### Bonus Assignment

#### Find all anagrams among words

Can you write an algorithm that finds all the anograms (a word, phrase, or name formed by rearranging the letters of another, such as `cinema`, formed from `iceman`) in the text?


#### Your Solution

In [ ]:
# TODO







### Conclusion

In this lab, we saw how an RDD of Tuple types receives via an implicit additional methods from PairRDDFunctions to make calculating certain values easier.

### Code as one continuous segment

Here is a possible solution to the discussion above written as one code segment.

In [ ]:
from operator import add
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Python Pair RDD Functions").getOrCreate()

file = "/home/jovyan/Resources/sherlock-holmes.txt"
text = spark.sparkContext.textFile(file)

import re

# use regular expression to split lines into words and translate them into lower case
words = text.flatMap(lambda w : re.split('[ \],.:;?!\-@#\(\)\\\*\"\/]*', w)).map(lambda w : w.lower())

# map each word to a pair with word as key and initial count 1
# then reduce by key getting the total counts for each word
pairs = words.map(lambda w : (w, 1)).reduceByKey(add)

# collect pairs into dictionary for random access
list = pairs.collectAsMap()

# print the counts for given words
print("if  count " + str(list["if"]))
print("and count " + str(list["and"]))
print("but count " + str(list["but"]))


### Solution for Bonus Assignment

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Python Pair RDD Functions").getOrCreate()

file = "/home/jovyan/Resources/sherlock-holmes.txt"
lines = spark.sparkContext.textFile(file)

import re

# use regular expression to split lines into words and translate them into lower case
words = lines.flatMap(lambda w : re.split('[ \],.:;?!\-@#\(\)\\\*\"\/]*', w)).map(lambda w : w.lower())

# select words longer than 6 letters and make them distinct
dstnc = words.filter(lambda w : len(w) > 6).distinct()

# we're creating a pair RDD where the key is the original word sorted by letters 
# and the initial value is a singleton list with the word as an element
# any two words sharing the same sorted forms are anagrams

pairs = dstnc.map(lambda w : (''.join(sorted(w)), [w]))

# the reduce function is concatenating lists of words
# map is dropping the keys as they're no loner needed
# filter is returning only those lists, which are longer than 1 (true anagrams)

angrs = pairs.reduceByKey(lambda w1, w2 : w1 + w2).map(lambda p : p[1]).filter(lambda w : len(w) > 1)

for a in angrs.collect() :
  print(a)